In [0]:
!pip install numpy pandas plotly surprise

# Filtrado Colaborativo
- Empareja usuarios con gente que tiene gustos similares
- Usuarios que tienen gustos similares se colocan en el mismo grupo/conjunto algorítmicamente hablando, y las recomendaciones se sugieren basadas en los gustos/intereses de estos usuarios en cada grupo particular (cluster)
- Presentaremos tres técnicas de **Filtrado Colaborativo**:  Filtrado Colaborativo usuario-usuario, Filtrado Colaborativo elemento-elemento y Factorización de Matrices

Para este fin, haremos uso de la librería [Surprise](http://surpriselib.com) de Python, una librería Python de código abierto fácil de usar para construir y analizar Sistemas de Recomendación, creada por *Nicolas Hug*.
Surprise proporciona la mayoría de los algoritmos fundamentales para crear Sistemas de Recomendación basados en Filtrado Colaborativo.
El nombre **SurPRISE** proviene de  **Simple Python RecommendatIon System Engine**.

Algunas de las características proporcionadas por Surprise son:
* Da al usuario un perfecto control sobre sus experimentos
* Documentación extensa y clara ofreciendo detalles precisos de cada algoritmo
* Una variedad de algoritmos de predicción listos para ser usados, como algoritmos baseline, métodos de vecindad (K-Neighbours), métodos basados en factorización de matrices (SVD, PMF, SVD++, NMF) y muchos más
* Diferentes medidas de similitud como Coseno, MSD (Mininum Square Difference) y Pearson entre otras
* Herramientas para evaluar, analizar y comparar el rendimiento de los algoritmos. Los procedimientos de validación cruzada pueden ser ejecutados muy fácilmente utilizando potentes iteradores de CV (inspirados en los que se pueden encontrar en scikit-learn), así como una búsqueda exhaustiva sobre un conjunto de parámetros
* Facilita la implementación de nuevas ideas de algoritmos de recomendación
* Alivia la tediosa tarea del manejo del conjunto de datos. Los usuarios pueden usar conjuntos de datos integrados (Movielens, Jester) y sus propios conjuntos de datos personalizados

En los siguientes ejemplos, utilizaremos el conjunto de datos *Movielens* para mostrar cómo construir, usar y evaluar algunos de los algoritmos de Filtrado Colaborativo. El conjunto de datos Movielens viene ya integrado en Surprise, permitiendo descargar diferentes conjuntos de datos dependiendo del tamaño deseado.
Movielens 100k (ml-100k), Movielens 1M (ml-1m) y Jester (jester) son los conjuntos de datos que vienen incorporados listos para usar en Surprise, de forma que no es necesario recoger, procesar y preparar datos de usuarios y elementos para comenzar a utilizar los algoritmos disponibles en Surprise.


# Análisis Exploratorio de Datos (EDA - Exploratory Data Analysis)

In [0]:
from surprise import Dataset, get_dataset_dir
import pandas as pd
#Load dataset using in-built Surprise Dataset class
Dataset.load_builtin('ml-100k')
user = pd.read_csv(get_dataset_dir()+ '/ml-100k/ml-100k/u.user', sep='|', error_bad_lines=False, encoding="latin-1")
user.columns = ['userID', 'age', 'gender', 'occupation', 'zipCode']
rating = pd.read_csv(get_dataset_dir() + '/ml-100k/ml-100k/u.data', sep='\t', error_bad_lines=False, encoding="latin-1")
rating.columns = ['userID', 'movieID', 'movieRating', 'timestamp']
df = pd.merge(user, rating, on='userID', how='inner')
df.drop(['age', 'gender', 'zipCode', 'timestamp'], axis=1, inplace=True)
df.head()

## Distribución de valoraciones

In [0]:
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

data = df['movieRating'].value_counts().sort_index(ascending=False)
print(data)
print(data.index)
print(data.values)
print(df.shape[0])

# Create the trace data
trace = go.Bar(x = data.index,
               text = ['{:.1f} %'.format(val) for val in (data.values / df.shape[0] * 100)],
               textposition = 'auto',
               textfont = dict(color = '#000000'),
               y = data.values,
               )
# Create figure layout
layout = dict(title = 'Distribución de {} valoraciones'.format(df.shape[0]),
              xaxis = dict(title = 'Valoración'),
              yaxis = dict(title = 'Recuento'))

# Create plot with data and layout
fig = go.Figure(data=[trace], layout=layout)
#iplot(fig)
fig.show(renderer='colab')


## Distribución de valoraciones por película
## Número de valoraciones por película

In [0]:
#Movies with rating bigger than 50, will be set to 50
data = df.groupby('movieID')['movieRating'].count().clip(upper=50)

# Create the trace data
trace = go.Histogram(x = data.values,
                     name = 'Valoraciones',
                     xbins = dict(start = 0,
                                  end = 51,
                                  size = 2))
# Create figure layout
layout = go.Layout(title = 'Distribución de número de valoraciones por película (Límite de 50)',
                   xaxis = dict(title = 'Número de valoraciones por película'),
                   yaxis = dict(title = 'Recuento'),
                   bargap = 0.2)

# Create plot with data and layout
fig = go.Figure(data=[trace], layout=layout)
#iplot(fig)
fig.show(renderer='colab')


print("Recuento Película-Puntuación (Top 10)")
movie_ratings_count = df.groupby('movieID')['movieRating'].count().reset_index().sort_values('movieRating', ascending=False)[:10]
print(movie_ratings_count)

print("Número de películas: %s" % df.groupby('movieID')['movieRating'].count().shape[0])



## Distribución de valoraciones por usuario

In [0]:
# Number of ratings per user
#Users which gave more than 50 ratings, will be cut off to 50
data = df.groupby('userID')['movieRating'].count().clip(upper=50)

# Create the trace data
trace = go.Histogram(x = data.values,
                     name = 'Valoraciones',
                     xbins = dict(start = 0,
                                  end = 51,
                                  size = 2))
# Create figure layout
layout = go.Layout(title = 'Distribución de número de valoraciones por usuario (Límite de 50)',
                   xaxis = dict(title = 'Valoraciones por usuario'),
                   yaxis = dict(title = 'Recuento'),
                   bargap = 0.2)

# Create plot with data and layout
fig = go.Figure(data=[trace], layout=layout)
#iplot(fig)
fig.show(renderer='colab')


print("Recuento Usuario-Puntuación (Top 10)")
user_ratings_count = df.groupby('userID')['movieRating'].count().reset_index().sort_values('movieRating', ascending=False)[:10]
print(user_ratings_count)

print("Número de usuarios: %s" % df.groupby('userID')['movieRating'].count().shape[0])

# Filtrado Colaborativo Usuario-Usuario (K-Nearest Neighbours Clustering)
- Para predecir la valoración de un usuario **u** sobre un elemento **i** (no valorado por el usuario u), usaremos la media ponderada (ajustada por la media de las valoraciones de cada usuario)
$$ p_{u,i} = \bar{r_u} + \frac{\sum_{v\in{N_i^{k}(u)}} sim(u,v) \cdot  (r_{vi} - \bar{r_v})}{\sum_{v\in{N_i^{k}(u)}} sim(u,v)}$$
- La salida es la predicción de la valoración del usuario u sobre el elemento i
- Usaremos la **medida de similitud entre el usuario u y el usuario v**, donde cada usuario v está en el mismo grupo (cluster) que el usuario u, calculado por el algoritmo
    - Como medida de similitud utilizaremos **Pearson baseline** 


In [0]:
from __future__ import (absolute_import, division, print_function, unicode_literals)  # Compatibility imports
import io

from collections import defaultdict

from surprise import Dataset, get_dataset_dir
from surprise import KNNWithMeans
from surprise import accuracy
from surprise.model_selection import train_test_split

Cargamos el conjunto de datos Movielens 100k (ml-100k) -> El formato es `UserID   MovieID Rating  Timestamp`

A continuación, dividimos el conjunto de datos en dos subconjuntos: conjunto de entrenamiento (training set) y conjunto de prueba (test set). Indicaremos que queremos utilizar el 15% de los datos para el conjunto de prueba

In [0]:
data = Dataset.load_builtin('ml-100k')
training_set, test_set = train_test_split(data, test_size=.15)
data.raw_ratings
def read_item_names():
    """Read the u.item file from MovieLens 100-k dataset and return two
    mappings to convert raw ids into movie names and movie names into raw ids.
    """

    file_name = get_dataset_dir() + '/ml-100k/ml-100k/u.item'
    rid_to_name = {}
    name_to_rid = {}
    with io.open(file_name, 'r', encoding='ISO-8859-1') as f:
        for line in f:
            line = line.split('|')
            rid_to_name[line[0]] = line[1]
            name_to_rid[line[1]] = line[0]

    return rid_to_name, name_to_rid

# Read the mappings raw id <-> movie name
rid_to_name, name_to_rid = read_item_names()

# Retrieve inner id of the movie Toy Story
#toy_story_raw_id = name_to_rid['Toy Story (1995)']
#toy_story_inner_id = algo.trainset.to_inner_iid(toy_story_raw_id)

Creamos una instancia del [algoritmo KNNWithMeans](https://surprise.readthedocs.io/en/stable/knn_inspired.html#surprise.prediction_algorithms.knns.KNNWithMeans) para calcular las predicciones basadas en los K-Vecinos más cercanos a un usuario dado.
El parámetro *sim_options* permite configurar la medida de similitud a utilizar y si el algoritmo debería utilizar un enfoque basado en *usuario* (calcula usuarios cercanos) o basado en *elemento* (calcula elementos cercanos). Se utiliza verdadero o falso para cambiar entre el enfoque de filtrado colaborativo basado en usuario y el basado en elemento
Como medida de similitud, vamos a emplear [Pearson Baseline](https://surprise.readthedocs.io/en/stable/similarities.html#surprise.similarities.pearson_baseline)


In [0]:
user_based_algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})

Una vez que hemos creado el algoritmo, necesitamos entrenarlo utilizando el conjunto de datos de entrenamiento.
Para ello, se utiliza el método *fit* de la instancia del algoritmo.

In [0]:
user_based_algo.fit(training_set)

Después de que el algoritmo haya sido entrenado (es decir, ha calculado las similitudes entre los usuarios y ha formado los grupos/clusters de usuarios), podemos pedirle por sugerencias/predicciones específicas.
Vamos a intentar predecir la valoración del elemento 302 (L.A. Confidential 1997) para el usuario 196 ( sabemos que el usuario valoraría esa película con un 4)
Sólo necesitamos llamar al método *predict* de la instancia del algoritmo, pasándole como parametros el usuario y el elementopara el cual queremos obtener la predicción calculada. 

In [0]:
user_id = str(196)
item_id = str(302)
print("Obteniendo la predicción del usuario 196 para la película", rid_to_name[item_id])

pred = user_based_algo.predict(user_id, item_id, r_ui=4, verbose=True)

print("La predicción es {}".format(pred.est))


Esto es sólo un ejemplo de como obtener una única predicción calculada para un usuario dado.
Si quisieramos obtener la lista de elementos recomendados para cierto usuario (elementos que no han sido valorados aún por el usuario),
Surprise no proporciona ninguna caracerística lista para usar para alcanzar este objetivo, pero podemos lograrlo utilizando características ya existentes y proporcionadas por Surprise. 

In [0]:
# Predict ratings for all pairs (u, i) that are NOT in the training set.
number_of_predictions = 10
no_ratings_set = training_set.build_anti_testset()
predictions = user_based_algo.test(no_ratings_set)
# First map the predictions to each user.
top_n = defaultdict(list)
for uid, iid, true_r, est, _ in predictions:
    top_n[uid].append((iid, est))

# Then sort the predictions for each user and retrieve the k highest ones.
for uid, user_ratings in top_n.items():
    user_ratings.sort(key=lambda x: x[1], reverse=True)
    top_n[uid] = user_ratings[:number_of_predictions]

# Print recommended items for user 196
for (iid, rating) in top_n[user_id]:
    print("Película: {} - Valoración: {}".format(rid_to_name[iid], rating))

De manera similar, también podemos evaluar cuán buenas son las predicciones calculadas.
Podemos emplear el modelo entrenado contra el conjunto de pruebas, utilizando el método *test*.
Después, podemos utilizar la raíz del error cuadrático medio - RMSE (Root Mean Square Error) - para evaluar cómo de bueno/malo es el modelo generado.
Surprise proporciona esa métrica de evaluación en su paquete **accuracy** (paquete de precisión) 

In [0]:
test_pred = user_based_algo.test(test_set)
print("Modelo basado en usuario : Conjunto de prueba")
accuracy.rmse(test_pred, verbose=True)

Únicamente utilizando el valor anterior de RMSE, normalmente no podremos saber/determinar si el modelo es bueno o malo por sí mismo.
Siendo ese el caso, una manera posible de saber como se comporta el modelo es mediante el cálculo de RMSE sobre el conjunto de entrenamiento y comparar dicho valor con el valor RMSE obtenido para el conjunto de prueba. 


In [0]:
print("Modelo basado en usuario : Conjunto de entrenamiento")
train_pred = user_based_algo.test(training_set.build_testset())
accuracy.rmse(train_pred)

Por regla general, si ambos valores de RMSE son similares, podríamos decir que nuestro modelo ha aprendido a predecir buenos valores y, por lo tanto, proporcionar buenas sugerencias.

Por otra parte, si el valor de RMSE del conjunto de prueba es mucho mayor que el que el valor RMSE del conjunto de entrenamiento probablemente el modelo erróneamente se ha sobreajustado a los datos (overfitting),
lo cual significa que hemos creado un modelo que cumple bien con los datos de ejemplo/prueba pero que posee escaso valor predictivo cuando se pone a prueba fuera de ese conjunto de prueba.
Por el contrario, si el valor de RMSE del conjunto de prueba es mucho más bajo, entonces el modelo subre de subajuste de datos (underfitting).

Sólo porque los valores de RMSE sean similares y nuestro modelo no está sobreajustando/subajustando los datos no quiere decir que hayamos construído un buen modelo. Sólamente indica que hemos construído un modelo que funciona consistentemente bien sobre los nuevos datos proporcionados.
Lo mejor que podemos hacer aquí es usar diferentes conjuntos de prueba y conjuntos de validación cruzada o cross-validation (datos que no están incluídos en los conjuntos de entrenamiento y prueba) para calcular más valores para ser comparados y utilizar métricas diferentes (y comparar dichos valores por ellos mismos).

De todas maneras, en esta situación, necesitamos desarrollar nuestra propia intuición aplicando esto y aprendiendo de muchos casos de uso diferentes.

Además, necesitamos ser conscientes que el valor de RMSE es dependiente de la escala, es decir, dependiente de nuestra variable dependiente (la escala de valoraciones en este caso).
Por esta razón, muchas soluciones utilizan el valor normalizado RMSE (NRMSE) utilizando los valores máximos y minimos o la media de los valores para hacerlos independiente de la escala y de alguna manera comparable entre modelos.


# Filtrado Colaborativo Elemento-Elemento
- Este método de Filtrado Colaborativo es similar al anterior con la diferencia de que la similitud se calcula entre elementos en vez de entre usuarios.

$$ p_{u,i} = \bar{r_i} + \frac{\sum_{j\in{N_u^{k}(i)}} sim(i,j) \cdot  (r_{uj} - \bar{r_j})}{\sum_{v\in{N_u^{k}(i)}} sim(i,j)}$$

- Observar que en la ecuación, **la similitud es entre el elemento i y j**, en lugar de entre el usuario u y v as como fue el caso en el [Filtrado Colaborativo Usuario-Usuario](#Filtrado-Colaborativo-Usuario-Usuario-(K-Nearest-Neighbours-Clustering))

- Algunas ventajas de los filtrados basados en elementos sobre los filtrados basados en usuarios son:
    - Escalan mejor. La razón es que el filtrado basado en usuario no escala muy bien porque los intereses (gustos) del usuario pueden cambiar frecuentemente dependiendo de, por ejemplo, la época del año como las campañas del Black Friday, Navidad, etc donde los usuarios pueden cambiar sus intereses buscando cosas muy distintas comparado con lo que suelen estar interesados. Debido a esto, los motores de recomendación necesitan ser re-entrenados con bastante frecuencia.
    - Más baratos computacionalmente hablando. En algunas situaciones, hay muchísimos más usuarios que elementos. En este caso, el filtrado colaborativo basado en elementos es más apropiado.
    
- Uno de los filtrados colaborativos basados en elementos más conocido es el [Motor de Recomendación de Amazon](#https://www.cs.umd.edu/~samir/498/Amazon-Recommendations.pdf)

In [0]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

# Built-in movielens-100k dataset.  UserID   MovieID Rating  Timestamp
data = Dataset.load_builtin('ml-100k')
training_set, test_set = train_test_split(data, test_size=.15)

# We want to run the algorithm based on items, se we configure the user_based option of the algorithm to false
item_based_algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})
item_based_algo.fit(training_set)

# Test the trained model against the test_set
test_pred = item_based_algo.test(test_set)

# Calculate RMSE of test predictions
print("Modelo basado en elemento : Conjunto de prueba")
accuracy.rmse(test_pred, verbose=True)

# Calculate RMSE of the training predictions
print("Modelo basado en elemento : Conjunto de entrenamiento")
training_pred = item_based_algo.test(training_set.build_testset())
accuracy.rmse(training_pred)

## Factorización de Matrices
Los métodos basados en usuarios y elementos son fáciles, intuitivos y comprensibles, pero en muchos casos no funcionan tan bien como podríamos esperar.

Otro método dentro de los métodos de Filtrado Colaborativo es la Factorización de Matrices (Matrix Factorization).
Las técnicas de factorización de matrices son generalmente más efectivas que las basadas en usuarios/elementos ya que nos permiten averiguar características latentes subyacentes a las interacciones entre usuarios y elementos, las cuales son por lo general, desconocidas para nosotros.

Una de las técnicas más populares de Factorización de Matrices es la *Descomposición de Valores Singulares*, **Singular Value Decomposition (SVD)**.
La técnica se hizo más popular después de la competición que propuse [Netflix Prize competition](https://en.wikipedia.org/wiki/Netflix_Prize). La entrada ganadora del famoso [Premio Netflix](https://www.netflixprize.com/) tenía una serie de modelos SVD, incluyendo SVD++ mezclado con máquinas Boltzmann restringidas (Restricted Boltzmann Machines).
Al utilizar estos métodos, el equipo ganador logró un aumento del 10 por ciento en precisión sobre el algoritmo existente de Netflix, ganando el premio de 1 millón de dólares.

El artículo [Netflix Prize and SVD](http://buzzard.ups.edu/courses/2014spring/420projects/math420-UPS-spring-2014-gower-netflix-SVD.pdf) diseñado para personas con un conociiento básico de álgebra lineal y descomposición, trata de explicar el funcionamiento de estos algoritmos de SVD de una manera que la mayoría de personas puedan entender.

Aunque esta técnica es mucho más compleja y difícil de entender, de nuevo, **Surprise** ha hecho todo el trabajo duro, de forma que nosotros podemos aplicar esta técnica fácilmente de una manera similar a la que ya hemos hecho para los algoritmos de filtrado colaborativo basado en usuarios y elementos.
 
En el siguiente ejemplo, se muestra como aplicar Factorización de Matrices por medio de SVD, haciendo uso del descenso de gradiente para minimizar el error cuadrático entre la valoración predicha y la real, obteniendo mejores modelos la mayoría de las veces.
Visitando la [documentación](http://surprise.readthedocs.io/en/stable/matrix_factorization.html) de Surprise, podemos aprender más acerca de las técnicas de Factorización de Matrices.

In [0]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

# Built-in movielens-100K dataset
data = Dataset.load_builtin('ml-100k')
training_set, test_set = train_test_split(data, test_size=.15)

# Singular Value Decomposition specific configuration
svd_algo = SVD(n_factors=160, n_epochs=100, lr_all=0.005, reg_all=0.1)
svd_algo.fit(training_set)

# Calculate predictions
test_pred = svd_algo.test(test_set)

# Calculate RMSE of test predictions
print("SVD : Conjunto de prueba")
accuracy.rmse(test_pred, verbose=True)

# Calculate RMSE of training predictions
print("SVD : Conjunto de entrenamiento")
training_pred = svd_algo.test(training_set.build_testset())
accuracy.rmse(training_pred)

En el ejemplo anterior, hemos configurado el algoritmo SVD con algunos parámetros, porque ya sabíamos por experimentación los mejores valores para ello.
Generalmente, la combinación de parámetros adecuada para nuestro caso de uso es desconocida y tenemos que hacer experimentaciones antes de decidir cuáles son los mejores valores de parámetros para el algoritmo.

Afortunadamente, **Surprise** contiene un conjunto de funciones de ayuda y procedimientos de Validación Cruzada que nos pueden ayudar a descubrir la mejor configuración para nuestros algoritmos.

La función **cross_validate()** y los iteradores de Validación Cruzada, proporcionan métricas de precisión sobre un procedimiento de validación cruzada para un conjunto de parámetros dado. 
Pero si lo que nosotros queremos es saber la combinación de parámetros que genera mejores resultados, la clase **GridSearchCV** es muy útil para este propósito. Dado un diccionario de parámetros, esta clase exhaustivamente prueba todas las combinaciones de parámetros e informa de los mejores parámetros para cualquier medida de precisión (ponderada sobre diferentes divisiones). **GridSearchCV** utiliza el iterador de Validación Cruzada [KFold](https://surprise.readthedocs.io/en/stable/model_selection.html#surprise.model_selection.split.KFold) internamente.  

In [0]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import GridSearchCV, cross_validate

# Built-in movielens-100K dataset
data = Dataset.load_builtin('ml-100k')

param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}

# GridSearchCV using RMSE and MAE
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

# Fit method to calculate the best configuration
gs.fit(data)

# Best RMSE score
print("Mejor valor de RMSE:", gs.best_score['rmse'])

# Get the best combination of parameters that gave the best RMSE score
print("Mejor combinación de parametros:", gs.best_params['rmse'])

# Now, we can obtain an algorithm instance that yields the best RMSE
best_svd_algo = gs.best_estimator['rmse']

# Let's run the cross_validate function against the best SVD algorithm obtained
print("\nValidación Cruzada del algoritmo SVD sobre 5 divisiones")
cross_validate(best_svd_algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# We can use the best SVD  algorithm to learn and create a model using fit
#best_svd_algo.fit(data.build_full_trainset())